# Regression Assignment for ECON 8877 in the OSU

## 1.First, we test OLS with homoskedasticity

### (a) For each row in the data, generate an error $\epsilon_{it}$ drawn iid from $N(0,7.5)$ (where 7.5 is the standard deviation, not variance).

### (b) Next, generate a column of data called "Choice" using the formula

\begin{align*}
y_{it}=\beta_{0}+\beta_{1}T_{i}+\beta_{2}S_{i}+\beta_{3}W_{i}+\beta_{4}G_{i}+\beta_{5}X_{t}+\epsilon_{it}.
\end{align*}

Use the following "true" values for $\beta$:
\begin{align*}
\beta = (\beta_{0},\dots,\beta_{5})=(1,0.75,0,0.10,0,-0.10)
\end{align*}

### (c) Run a regression to get $\hat{\beta}$. Note the $p$-values as well.

### (d) Repeat the previous three steps 100 times.

### (e) For each $\beta_{j}$, what is the median value of $\hat{\beta}_{j}$ (out of 100)? How close it to the true value of $\beta_{j}$? This is an estimate of the bias in your regression.

### (f) For each $\beta_{j}$, count the fraction of the 100 regressions for which the $p$-value was below 0.05. This is the estimated power of that test.

We can deal with the problems above ((a)$\sim$(f)) with the code below.

We first make an environment for the project and add packages that are relevant. Also, activate all the packages.

In [2]:
using Pkg
pkg"activate @reg_env" # we create a new shared environment 
pkg"add CSV";pkg"add DataFrames";pkg"add MLBase";pkg"add Random";pkg"add LinearAlgebra";pkg"add Distributions";pkg"add GLM"
pkg"add CovarianceMatrices";pkg"add BrowseTables";
using CSV, DataFrames, MLBase, Random, LinearAlgebra, Distributions, GLM, CovarianceMatrices, BrowseTables



┌ Warning: The Pkg REPL mode is intended for interactive use only, and should not be used from scripts. It is recommended to use the functional API instead.
└ @ Pkg.REPLMode C:\Users\LG\AppData\Local\Programs\julia-1.9.4\share\julia\stdlib\v1.9\Pkg\src\REPLMode\REPLMode.jl:382


  Activating

 project at `C:\Users\LG\.julia\environments\reg_env`


    Updating registry at `C:\Users\LG\.julia\registries\JuliaComputingRegistry`
    Updating registry at `C:\Users\LG\.julia\registries\General.toml`


   Resolving package versions...

...

   Resolving package versions...


  No Changes to `C:\Users\LG\.julia\environments\reg_env\Project.toml`
  No Changes

 to `C:\Users\LG\.julia\environments\reg_env\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\LG\.julia\environments\reg_env\Project.toml`
  No Changes to `C:\Users\LG\.julia\environments\reg_env\Manifest.toml`

Then, make a function that make the csv file into a data frame and define the variables with names.

In [3]:
function environment()
    df = DataFrame(CSV.File("fakedata.csv"))
    SubjectID = df[:,1];
    DecisionID = df[:,2];
    Constant = df[:,3];
    Treatment = df[:,4];
    SessionID = df[:,5];
    SwitchPoint = df[:,6];
    Gender = df[:,7];
    Complexity = df[:,8];
    return SubjectID, DecisionID, Constant, Treatment, SessionID, SwitchPoint, Gender, Complexity
end

environment (generic function with 1 method)

Next, we make the function that conducts the main simulation.

In [4]:
function First(rep_time)
    SubjectID, DecisionID, Constant, Treatment, SessionID, SwitchPoint, Gender, Complexity = environment();
    n_i=length(unique(SubjectID));n_t=length(unique(DecisionID));
    X = [Constant Treatment SessionID SwitchPoint Gender Complexity];
    β = [1.0; 0.75; 0.0; 0.01; 0.0; -0.1]; n_β = length(β);
    β̂_Mat_builtin = zeros(n_β,rep_time);
    p_val_Mat_builtin = zeros(n_β,rep_time);
    for s = 1:rep_time
        ϵ = rand(Normal(0,7.5),n_i*n_t);
        Choice = X * β + ϵ; df=(;Choice,Treatment,SessionID, SwitchPoint, Gender, Complexity);
        lm_builtin = lm(@formula(Choice~Treatment+SessionID+SwitchPoint+Gender+Complexity),df);
        p_val_Mat_builtin[:,s] = coeftable(lm_builtin).cols[4];
        β̂_Mat_builtin[:,s] = coeftable(lm_builtin).cols[1];
    end # end from 1.(a) to 1.(d)
    median_β̂_builtin = median(β̂_Mat_builtin, dims=2);
    power_β̂_builtin = sum(p_val_Mat_builtin.<0.05, dims=2)./rep_time # 1.(f)    
    return β̂_Mat_builtin, p_val_Mat_builtin, median_β̂_builtin, power_β̂_builtin
end

β̂_Mat1, p_val_Mat1, median_β̂1, power_β̂1 = First(1000)

println(median_β̂1.-[1.0; 0.75; 0.0; 0.01; 0.0; -0.1])
println(power_β̂1)

[0.022213276922260716; 0.010695286686625227; -0.0037565193334643063; -0.002449517470004087; 0.0035036821911714313; 0.0030857752890625367;;]
[0.351; 0.193; 0.039; 0.059; 0.047; 0.659;;]


We can see that the defined biases are quite small. However, the power of the test is very low. For example, the power of $β_{3}$ is almost similar to the rejection rate of $β_{2}$ and $β_{4}$, whose true values are 0. In return, Type I error is less occurring, i.e., almost exact.

Now let's move on to the second question.

## 2.Re-run that exercise, but with heterskedasticity. Specifically, let the standard deviation of $ϵ_{it}$ equal to the Decision ID number. So, for example, everyone whose choice on decision 8 has noise $ϵ_{it}\sim N(0,8)$. Re-generate these errors with that structure, and then re-generate the Choice data using the same linear equation as above.

### 2.(a) This time, run 100 regressions of OLS, then 100 each with HC0, HC1, HC2, and HC3.

In [5]:
function pnorm_std(x::Real) #Since I will make the manual code fot the test, we need to define this
    p = cdf(Normal(0,1),x)
    return p
end

function Second(rep_time)
    SubjectID, DecisionID, Constant, Treatment, SessionID, SwitchPoint, Gender, Complexity = environment();
    n_i=length(unique(SubjectID));n_t=length(unique(DecisionID));
    X = [Constant Treatment SessionID SwitchPoint Gender Complexity];
    β = [1.0; 0.75; 0.0; 0.01; 0.0; -0.1]; n_β = length(β);
    β̂_Mat = zeros(n_β,rep_time);
    ϵ = zeros(n_i*n_t);
    p_val_Mat_built = zeros(n_β,rep_time);
    p_val_Mat_HC0 = zeros(n_β,rep_time);
    p_val_Mat_HC1 = similar(p_val_Mat_HC0);
    p_val_Mat_HC2 = similar(p_val_Mat_HC0);
    p_val_Mat_HC3 = similar(p_val_Mat_HC0);
    for s = 1:rep_time
        for j = 1:(n_i*n_t)
            ϵ[j] = rand(Normal(0,DecisionID[j]))
        end
        Choice = X * β + ϵ; df=(;Choice,Treatment,SessionID, SwitchPoint, Gender, Complexity);
        β̂  = (X' * X) \ (X' * Choice); Choicê = X * β̂ ; ϵ̂  = Choice .- Choicê;
        Var_β̂_HC0 = inv(X' * X) * X' * Diagonal(diag(ϵ̂  * ϵ̂' )) * X * inv(X' * X);
        Var_β̂_HC1 = Var_β̂_HC0 .* (n_i*n_t) ./ (n_i*n_t - n_β);
        Var_β̂_HC2 = inv(X' * X) * X' * Diagonal(diag(ϵ̂  * ϵ̂') ./ (1 .- diag(X*inv(X'*X)*X'))) * X * inv(X' * X);
        Var_β̂_HC3 = inv(X' * X) * X' * Diagonal(diag(ϵ̂  * ϵ̂') ./ ((1 .- diag(X*inv(X'*X)*X')).^2)) * X * inv(X' * X);
        t_val_HC0 = β̂  ./ sqrt.(diag( Var_β̂_HC0 ));
        t_val_HC1 = β̂  ./ sqrt.(diag( Var_β̂_HC1 ));
        t_val_HC2 = β̂  ./ sqrt.(diag( Var_β̂_HC2 ));
        t_val_HC3 = β̂  ./ sqrt.(diag( Var_β̂_HC3 ));
        lm_builtin = lm(@formula(Choice~Treatment+SessionID+SwitchPoint+Gender+Complexity),df);
        for l = 1:n_β
            p_val_Mat_HC0[l,s] = ifelse(t_val_HC0[l]>0, 1-pnorm_std(t_val_HC0[l])+pnorm_std(-t_val_HC0[l]), pnorm_std(t_val_HC0[l])+1-pnorm_std(-t_val_HC0[l]));
            p_val_Mat_HC1[l,s] = ifelse(t_val_HC1[l]>0, 1-pnorm_std(t_val_HC1[l])+pnorm_std(-t_val_HC1[l]), pnorm_std(t_val_HC1[l])+1-pnorm_std(-t_val_HC1[l]));
            p_val_Mat_HC2[l,s] = ifelse(t_val_HC2[l]>0, 1-pnorm_std(t_val_HC2[l])+pnorm_std(-t_val_HC2[l]), pnorm_std(t_val_HC2[l])+1-pnorm_std(-t_val_HC2[l]));
            p_val_Mat_HC3[l,s] = ifelse(t_val_HC3[l]>0, 1-pnorm_std(t_val_HC3[l])+pnorm_std(-t_val_HC3[l]), pnorm_std(t_val_HC3[l])+1-pnorm_std(-t_val_HC3[l]));
        end
        p_val_Mat_built[:,s] = coeftable(lm_builtin).cols[4]
        β̂_Mat[:,s] = β̂ 
    end # end from 1.(a) to 1.(d)
    median_β̂  = median(β̂_Mat, dims=2); # 1.(e)
    power_β̂_builtin = sum(p_val_Mat_built.<0.05, dims=2)./rep_time
    power_β̂_HC0  = sum(p_val_Mat_HC0.<0.05, dims=2)./rep_time
    power_β̂_HC1  = sum(p_val_Mat_HC1.<0.05, dims=2)./rep_time
    power_β̂_HC2  = sum(p_val_Mat_HC2.<0.05, dims=2)./rep_time
    power_β̂_HC3  = sum(p_val_Mat_HC3.<0.05, dims=2)./rep_time
    p_val_Mat = [p_val_Mat_built;;;p_val_Mat_HC0;;;p_val_Mat_HC1;;;p_val_Mat_HC2;;;p_val_Mat_HC3]
    return β̂_Mat, p_val_Mat, median_β̂ , power_β̂_builtin, power_β̂_HC0, power_β̂_HC1, power_β̂_HC2, power_β̂_HC3
end

@time β̂_Mat2, p_val_Mat2, median_β̂2 , power_β̂_builtin2, power_β̂_HC02, power_β̂_HC12, power_β̂_HC22, power_β̂_HC32 = Second(100)

powerdata=DataFrame(nonrobust=vec(power_β̂_builtin2),HC0=vec(power_β̂_HC02),HC1=vec(power_β̂_HC12),HC2=vec(power_β̂_HC22),HC3=vec(power_β̂_HC32));
println(powerdata);
open_html_table(powerdata);

 10.019546 seconds (6.06 M allocations: 12.590 GiB, 9.98% gc time, 30.83% compilation time)


6×5 DataFrame
 Row │ nonrobust  HC0      HC1      HC2      HC3     
     │ Float64    Float64  Float64  Float64  Float64 
─────┼───────────────────────────────────────────────
   1 │      0.26     0.35     0.35     0.35     0.35
   2 │      0.14     0.15     0.15     0.15     0.15
   3 │      0.04     0.04     0.04     0.04     0.03
   4 │      0.04     0.04     0.04     0.04     0.04
   5 │      0.04     0.04     0.04     0.04     0.04
   6 │      0.5      0.5      0.5      0.5      0.5

### 2.(b) Which methods give unbiased estimates?

Since regression is the same for all the methods, they are all unbiased estimates.

### 2.(c) Among methods with unbiased estimates, which has the most power (when $\beta_{j}\neq 0$).

I felt that it is a little vague to check this when there is only 100 repetition. So I have chosen to do this with 1000 repetition. As can be seen from the power table, within heteroskedasticity robust tests, HC0 has the largest power and power keeps go down from HC1 to HC3. When compared to non-robust one, for some $\beta$, non-robust reject more and for the others, robust tests reject more.

## 3.Now let's add session effects. The way to do this is first create a random column $ϵ_{it}\sim N(0,7.5)$ of uncorrelated errors. Then, for each session $S$, generate a \textit{single} random number $u_{S}\sim N(0,7.5)$. Finally, generate the Choice variable using the linear equation

\begin{align*}
y_{it}=\beta_{0}+\beta_{1}T_{i}+\beta_{2}S_{i}+\beta_{3}W_{i}+β_{4}G_{i}+\beta_{5}X_{t}+u_{S_{i}}+ϵ_{it}.
\end{align*}
Use the same values for $\beta$ as above.

### 3.(a) Run 100 regressions of OLS, ignoring the session effects. Are your estimates biased? Is your power affected? Are the tests still valid (rejecting 5% of the time when $\beta=0$)? Pay special attention to $\hat{\beta}_{2}$, which measures session effects directly.

In [6]:
function Third_a(rep_time)
    SubjectID, DecisionID, Constant, Treatment, SessionID, SwitchPoint, Gender, Complexity = environment();
    n_i=length(unique(SubjectID));n_t=length(unique(DecisionID));n_s=length(unique(SessionID));
    X = [Constant Treatment SessionID SwitchPoint Gender Complexity];
    β = [1.0; 0.75; 0.0; 0.01; 0.0; -0.1]; n_β = length(β);
    β̂_Mat = zeros(n_β,rep_time);
    p_val_Mat_builtin = zeros(n_β,rep_time);
    for s = 1:rep_time
        ϵ = rand(Normal(0,7.5),n_i*n_t);u = repeat(rand(Normal(0,7.5),n_s),inner=Int64(n_i*n_t/n_s));
        Choice = X * β + ϵ + u; df=(;Choice,Treatment,SessionID, SwitchPoint, Gender, Complexity);
        lm_builtin = lm(@formula(Choice~Treatment+SessionID+SwitchPoint+Gender+Complexity),df);
        p_val_Mat_builtin[:,s] = coeftable(lm_builtin).cols[4]
        β̂_Mat[:,s] = coeftable(lm_builtin).cols[1]
    end # end from 1.(a) to 1.(d)
    bias = median(β̂_Mat, dims = 2) .- β;
    power_β̂_builtin = sum(p_val_Mat_builtin.<0.05, dims=2)./rep_time
    return β̂_Mat, bias, power_β̂_builtin
end

@time β̂_Mat3a, bias3a, power3a =Third_a(1000)

println(DataFrame(bias = vec(bias3a), power = vec(power3a)))

  0.880015 seconds (852.47 k allocations: 593.398 MiB, 14.21% gc time, 24.64% compilation time)
6×2 DataFrame
 Row │ bias          power   
     │ Float64       Float64 
─────┼───────────────────────
   1 │ -0.181968       0.692
   2 │ -0.390692       0.776
   3 │  0.0363352      0.778
   4 │ -0.00550963     0.476
   5 │ -0.00965869     0.01
   6 │ -0.000671038    0.43


The bias for $\beta_{1}$ keeps look large. But biases for other $β$ s are not very severe always, including $β_{2}$. The biggest problem seems to be the power of $β_{2}$. It should be close to 0.05 to be valid because the true value is 0, but the rejection rate is far more high.

### 3.(b) Run 100 regressions using with clustering at the session level. Does this fix any bias? Does it give better power? Are tests valid?

Let's describe how the clustered variance-covariance matrix will look like.

\begin{align*}
Cov(u_{S_{i}}+ϵ_{it},u_{S_{j}}+ϵ_{jt})=
\begin{cases}  
V(u_{S_{i}}),& i\neq j\:\&\:S_{i}=S_{j}\\
V(u_{S_{i}})+V(ϵ_{it}),& i=j\\
0, & o/w.
\end{cases}
\end{align*}
Then, in terms of estimator, we can use the method given in several textbooks and also used in Stata, which is 
\begin{align*}
    &\hat{V}_{\hat{\beta}}=a_{n}(X'X)^{-1}\hat{Ω}(X'X)^{-1}\\
    \text{where }&\hat{Ω}=\sum_{s=1}^{20}X_{s}'\hat{e}_{s}\hat{e}_{s}'X_{s}\text{ and }a_{n}=\frac{n-1}{n-k}\frac{S}{S-1}.
\end{align*}
Here, $S$ is the number of clusters.

In [7]:
function Third_b(rep_time)
    SubjectID, DecisionID, Constant, Treatment, SessionID, SwitchPoint, Gender, Complexity = environment();
    n_i=length(unique(SubjectID));n_t=length(unique(DecisionID));n_s=length(unique(SessionID));
    X = [Constant Treatment SessionID SwitchPoint Gender Complexity];
    β = [1.0; 0.75; 0.0; 0.01; 0.0; -0.1]; n_β = length(β);
    β̂_Mat = zeros(n_β,rep_time);
    p_val_Mat_cluster = zeros(n_β,rep_time);
    for s = 1:rep_time
        ϵ = rand(Normal(0,7.5),n_i*n_t);u = repeat(rand(Normal(0,7.5),n_s),inner=Int64(n_i*n_t/n_s));
        Choice = X * β + ϵ + u; df=(;Choice,Treatment,SessionID, SwitchPoint, Gender, Complexity);
        lm_builtin = lm(@formula(Choice~Treatment+SessionID+SwitchPoint+Gender+Complexity),df);
        β̂_Mat[:,s] = coeftable(lm_builtin).cols[1];
        resd = Choice .- fitted(lm_builtin);
        part_resd = collect(Iterators.partition(resd,Int64(n_i*n_t/n_s))); # partitioning the residuals wrt clusters
        part_X = [X[i:min(i+Int64(n_i*n_t/n_s)-1,Int64(n_i*n_t)),:] for i in 1:Int64(n_i*n_t/n_s):Int64(n_i*n_t)]; # partitioning the covariates wrt clusters
        Ω̂  = zeros(n_β,n_β);
        for j = 1:n_s
            X_g = part_X[j]
            resd_g = part_resd[j]
            Ω̂  = Ω̂  + X_g' * resd_g * resd_g' * X_g
        end
        a = (n_i*n_t-1)/(n_i*n_t-n_β)*n_s/(n_s-1);
        Var_β̂_cluster = a .* inv(X' * X) * Ω̂  * inv(X' * X);
        t_val_cluster = β̂_Mat[:,s] ./ sqrt.(diag( Var_β̂_cluster ));
        for l = 1:n_β
            p_val_Mat_cluster[l,s] = ifelse(t_val_cluster[l]>0, 1-pnorm_std(t_val_cluster[l])+pnorm_std(-t_val_cluster[l]), pnorm_std(t_val_cluster[l])+1-pnorm_std(-t_val_cluster[l]));
        end
    end # end from 1.(a) to 1.(d)
    bias = median(β̂_Mat, dims = 2) .- β;
    power_β̂_cluster = sum(p_val_Mat_cluster.<0.05, dims=2)./rep_time
    return β̂_Mat, bias, power_β̂_cluster
end

@time β̂_Mat3b, bias3b, power3b =Third_b(1000)

println(DataFrame(bias = vec(bias3b), power = vec(power3b)))

  1.422026 seconds (1.65 M allocations: 1.401 GiB, 11.42% gc time, 27.67% compilation time)
6×2 DataFrame
 Row │ bias         power   
     │ Float64      Float64 
─────┼──────────────────────
   1 │  0.0944639     0.11
   2 │ -0.230721      0.104
   3 │ -0.00305999    0.1
   4 │ -0.00298581    0.081
   5 │  0.00491917    0.057
   6 │ -0.00216239    0.705


Since we don't change the way we estimate the $β$ s, there is no systematical reason that bias should change. However, it is quite prominent that the power of the most of the coefficients has dropped significantly. But still it is not valid as can be seen from the table. Being not sure about my manual code for clustered variance-covariance matrix, I used the package "FixedEffectModels" to double check. 

In [8]:
pkg"add FixedEffectModels"
using FixedEffectModels

function Third_b_ver2(rep_time)
    SubjectID, DecisionID, Constant, Treatment, SessionID, SwitchPoint, Gender, Complexity = environment();
    n_i=length(unique(SubjectID));n_t=length(unique(DecisionID));n_s=length(unique(SessionID));
    X = [Constant Treatment SessionID SwitchPoint Gender Complexity];
    β = [1.0; 0.75; 0.0; 0.01; 0.0; -0.1]; n_β = length(β);
    β̂_Mat = zeros(n_β,rep_time);
    p_val_Mat_cluster = zeros(n_β,rep_time);
    for s = 1:rep_time
        ϵ = rand(Normal(0,7.5),n_i*n_t);u = repeat(rand(Normal(0,7.5),n_s),inner=Int64(n_i*n_t/n_s));
        Choice = X * β + ϵ + u; df=(;Choice,Treatment,SessionID, SwitchPoint, Gender, Complexity);
        lm_builtin = reg(df, @formula(Choice~Treatment+SessionID+SwitchPoint+Gender+Complexity), Vcov.cluster(:SessionID));
        β̂_Mat[:,s] = coef(lm_builtin);
        p_val_Mat_cluster[:,s] = coeftable(lm_builtin).cols[4];
    end # end from 1.(a) to 1.(d)
    bias = median(β̂_Mat, dims = 2) .- β;
    power_β̂_cluster = sum(p_val_Mat_cluster.<0.05, dims=2)./rep_time
    return β̂_Mat, bias, power_β̂_cluster
end

@time β̂_Mat3b_ver2, bias3b_ver2, power3b_ver2 = Third_b_ver2(1000)
# We have to adjust the order of rows to match since FixedEffectModels give intercept at the last row.
β̂_Mat3b_ver2 = vcat(β̂_Mat3b_ver2[size(β̂_Mat3b_ver2,1),:]',β̂_Mat3b_ver2);
β̂_Mat3b_ver2 = β̂_Mat3b_ver2[1:size(β̂_Mat3b_ver2,1)-1]
bias3b_ver2 = vcat(bias3b_ver2[size(bias3b_ver2,1),:]',bias3b_ver2);
bias3b_ver2 = bias3b_ver2[1:size(bias3b_ver2,1)-1,:];
power3b_ver2 = vcat(power3b_ver2[size(power3b_ver2,1),:]',power3b_ver2);
power3b_ver2 = power3b_ver2[1:size(power3b_ver2,1)-1,:];

println(DataFrame(bias = vec(bias3b_ver2), power = vec(power3b_ver2)))

   Resolving package versions...


  No Changes to `C:\Users\LG\.julia\environments\reg_env\Project.toml`
  No Changes

 to `C:\Users\LG\.julia\environments\reg_env\Manifest.toml`


  3.731852 seconds (5.43 M allocations: 650.115 MiB, 2.69% gc time, 82.90% compilation time)


6×2 DataFrame
 Row │ bias          power   
     │ Float64       Float64 
─────┼───────────────────────
   1 │ -0.00115347     0.109
   2 │ -0.0993736      0.095
   3 │ -0.391898       0.091
   4 │  0.0171049      0.054
   5 │  0.0086169      0.05
   6 │ -0.000587324    0.646


Now the result shows that the variance-covariance may not be very different from manual one because the characteristics of power doesn't seem to be very much different.

### 3.(c) Run 100 regressions using random effects. Note that the $u_{S}$ is highly correlated with one of the $X$ columns, so we're told this is not valid. Analyze bias, power, and validity.

Following the Bruce Hansen's textbook (2022), I manually coded the Random Effects model as below.

In [20]:
function Third_c_manual(rep_time)
    SubjectID, DecisionID, Constant, Treatment, SessionID, SwitchPoint, Gender, Complexity = environment();
    n_i=length(unique(SubjectID));n_t=length(unique(DecisionID));n_s=length(unique(SessionID));n_i_per_s=Int64(n_i*n_t/n_s)
    ObservationID = repeat([1:1:90;],n_s); # needed to fit into the panel data setting
    X = [Constant Treatment SessionID SwitchPoint Gender Complexity];
    β = [1.0; 0.75; 0.0; 0.01; 0.0; -0.1]; n_β = length(β);
    β̂_Mat = zeros(n_β,rep_time);
    p_val_Mat_RE = zeros(n_β,rep_time);
    for s = 1:rep_time
        ϵ = rand(Normal(0,7.5),n_i*n_t);u = repeat(rand(Normal(0,7.5),n_s),inner=n_i_per_s);
        Choice = X * β + ϵ + u; df=(;Choice,Treatment,SessionID, SwitchPoint, Gender, Complexity,SubjectID,ObservationID);
        lm_POLS = reg(df, @formula(Choice~Treatment+SessionID+SwitchPoint+Gender+Complexity));
        v̂ = residuals(lm_POLS,df);
        σ̂_v = sqrt(sum(v̂.^2)/(n_i*n_t-n_β));
        sum_of_cross_product = 0;
        for i = 1:n_s, j = 1:(n_i_per_s-1), k = (j+1):n_i_per_s
            sum_of_cross_product = sum_of_cross_product+v̂[n_i_per_s*(i-1)+j]*v̂[n_i_per_s*(i-1)+k]
        end
        σ̂_u = sqrt(sum_of_cross_product/(n_s*n_i_per_s*(n_i_per_s-1)/2-n_β))
        σ̂_ϵ = sqrt(σ̂_v^2 - σ̂_u^2)
        Ω̂_s  = σ̂_u^2 .* ones(n_i_per_s,n_i_per_s) .+ Diagonal(fill(σ̂_ϵ^2,n_i_per_s)); # covariance structure for each session
        Ω̂  = Symmetric(kron(Diagonal(ones(n_s)),Ω̂_s)) ; # covariance structure for the whole session
        β̂_RE  = inv(X' * inv(Ω̂ ) * X) * X' * inv(Ω̂ ) * Choice;
        resid_RE = Choice .- X * β̂_RE;
        V̂_RE =  inv(X' * inv(Ω̂ ) * X) .* σ̂_ϵ^2;
        t_val_vec_RE = β̂_RE ./ sqrt.(diag(V̂_RE))
        for l = 1:n_β
            p_val_Mat_RE[l,s] = ifelse(t_val_vec_RE[l]>0, 1-pnorm_std(t_val_vec_RE[l])+pnorm_std(-t_val_vec_RE[l]), pnorm_std(t_val_vec_RE[l])+1-pnorm_std(-t_val_vec_RE[l]));
        end
        β̂_Mat[:,s] = β̂_RE
    end
    bias = median(β̂_Mat, dims = 2) .- β;
    power_β̂_RE = sum(p_val_Mat_RE.<0.05, dims=2)./rep_time
    return β̂_Mat, bias, power_β̂_RE 
end

@time β̂_Mat3c, bias3c, power3c =Third_c_manual(500)

println(DataFrame(bias = vec(bias3c), power = vec(power3c)))


303.493250 seconds (316.15 M allocations: 55.420 GiB, 1.47% gc time)
6×2 DataFrame
 Row │ bias         power   
     │ Float64      Float64 
─────┼──────────────────────
   1 │ -0.145666        0.0
   2 │  0.128552        0.0
   3 │  0.0312888       0.0
   4 │ -0.00190415      0.0
   5 │ -0.0232606       0.0
   6 │ -0.00226533      0.0


It takes quite a while to calculate and also the result was bizarre. Powers of RE model here is 0. Also, bias is bigger than the session level controlled one.

### 3.(d) Run 100 regressions with fixed effects. Analyze bias, power, and validity.

I was not successful in conducting this simulation. This is mainly because my variance estimator for $\beta$ s keep made a negative values. Also, all the built-in codes also showed error messages when trying to use it. They tend to spit out error message about matrix inversion. The manual code I tried also show a problem of never rejecting the null. Moreover, the bias of the estimators are higher than the Random Effects model which seem off. So, probably, there is some problem in my code, but I don't know what is the problem. I'll just show my manual code as follows.

In [14]:
function Third_d_manual(rep_time)
    SubjectID, DecisionID, Constant, Treatment, SessionID, SwitchPoint, Gender, Complexity = environment();
    n_i=length(unique(SubjectID));n_t=length(unique(DecisionID));n_s=length(unique(SessionID));n_i_per_s=Int64(n_i*n_t/n_s)
    ObservationID = repeat([1:1:90;],n_s); # needed to fit into the panel data setting
    X = [Constant Treatment SessionID SwitchPoint Gender Complexity];
    β = [1.0; 0.75; 0.0; 0.01; 0.0; -0.1]; n_β = length(β);
    β̂_Mat = zeros(n_β,rep_time);
    p_val_Mat_FE = zeros(n_β,rep_time); 
    for s = 1:rep_time
        ϵ = rand(Normal(0,7.5),n_i*n_t);u = repeat(rand(Normal(0,7.5),n_s),inner=n_i_per_s);
        Choice = X * β + ϵ + u;
        I_s = ones(n_i_per_s)
        B = Symmetric(I_s'*I_s)
        M_s = Diagonal(I_s) .- I_s * inv(B) * I_s' # demean operator for a session
        M = kron(Diagonal(ones(n_s)),M_s) # demean operator for all sessions
        A = Symmetric(X' * M * X)
        β̂_FE = inv(A) * X' * M * Choice
        resid_FE = Choice .- X * β̂_FE
        part_resid_FE = collect(Iterators.partition(resid_FE,n_i_per_s)); # partitioning the residuals wrt clusters
        part_X = [X[i:min(i+n_i_per_s-1,n_i*n_t),:] for i in 1:n_i_per_s:(n_i*n_t)]; # partitioning the covariates wrt clusters
        Ω̂  = zeros(n_β,n_β);
        for j = 1:n_s
            X_g = part_X[j]
            resid_g = part_resid_FE[j]
            Ω̂  = Ω̂  + X_g' * resid_g * resid_g' * X_g
        end
        V̂_FE = inv(A) * Ω̂ * inv(A)
        β̂_Mat[:,s] = β̂_FE
        t_val_vec_FE = β̂_FE ./ sqrt.(diag(V̂_FE))
        for l = 1:n_β
        p_val_Mat_FE[l,s] = ifelse(t_val_vec_FE[l]>0, 1-pnorm_std(t_val_vec_FE[l])+pnorm_std(-t_val_vec_FE[l]), pnorm_std(t_val_vec_FE[l])+1-pnorm_std(-t_val_vec_FE[l]));
        end
    end
    bias = median(β̂_Mat, dims = 2) .- β;
    power_β̂_FE = sum(p_val_Mat_FE.<0.05, dims=2)./rep_time
    return β̂_Mat, bias, power_β̂_FE 
end

@time β̂_Mat3d_FEpack, bias3d_FEpack, power3d_FEpack =Third_d_manual(1000)

println(DataFrame(bias = vec(bias3d_FEpack), power = vec(power3d_FEpack)))

 27.409564 seconds (434.38 k allocations: 25.257 GiB, 4.97% gc time, 0.12% compilation time)
6×2 DataFrame
 Row │ bias         power   
     │ Float64      Float64 
─────┼──────────────────────
   1 │ -1.41388         0.0
   2 │ -2.4937          0.0
   3 │  0.175748        0.0
   4 │  0.00386557      0.0
   5 │  0.00254427      0.0
   6 │ -0.0024068       0.0


### 3.(e) Our regression equation contains $\beta_{2}S_{i}$, which assumes a linear relationship between the session ID and choices. This seems a bit silly. Instead, one could create dummy variables for each of the sessions. So, replace $S_{i}$ with 19 dummy variables (leaving session 1 as the omitted session). Run 100 regressions of regular OLS (with no clustering, fixed effects, or random effects) but with this dummy variable specification. Presumably this drastically reduces power since we've added so many variables. Analyze bias, power, and validity.

We not only exclude the dummy variable for group 1 but also exclude dummy variable for group 11 since if there were group 11 to group 20 dummy variables, the sum of the variables are exactly the same with the treatment variable which assigns 1 to group 11 to 20 and 0 to group 1 to 10.

Note that our true model is 
\begin{align*}
y_{it}=\beta_{0}+\beta_{1}T_{i}+\beta_{3}W_{i}+\beta_{4}G_{i}+\beta_{5}X_{t}+u_{S_{i}}+ϵ_{it},
\end{align*}
while model used for estimation is
\begin{align*}
y_{it}=\gamma_{0}+\gamma_{1}T_{i}+\gamma_{3}W_{i}+\gamma_{4}G_{i}+\gamma_{5}X_{t}+\sum_{j\neq 1,11}\delta_{j}D_{j}+ϵ_{it}.
\end{align*}

Thus, for $j=1$, the intercept of the group is $\beta_{0}+u_{1}=\gamma_{0}$, while for $j=2,\dots,10$, $\beta_{0}+u_{j}=\gamma_{0}+\delta_{j}$. Thus, $\delta_{j}=u_{j}-u_{1}$ for $j=2,\dots,10$. For $j=11$, the intercept of the group is $\beta_{0}+\beta_{1}+u_{11}=\gamma_{0}+\gamma_{1}$, while for $j=12,\dots,20$, the intercept is $\beta_{0}+\beta_{1}+u_{j}=\gamma_{0}+\gamma_{1}+\delta_{j}$, and hence $\delta_{j}=u_{j}-u_{11}$. Moreover, from $\beta_{0}+\beta_{1}+u_{11}=\gamma_{0}+\gamma_{1}=\beta_{0}+u_{1}+\gamma_{1}$, we can see that $\gamma_{1}=\beta_{1}+u_{11}-u_{1}$. Since $u_{j}$ are mean 0 random variable, the estimators for $\gamma_{0},\gamma_{1}$ can be unbiased estimator for $\beta_{0}$, $\beta_{1}$, each, but it is quite clear that variance of the estimators will be large because of the large standard deviations of $u_{j}$ s.

In [19]:
function Third_e_manual(rep_time)
    SubjectID, DecisionID, Constant, Treatment, SessionID, SwitchPoint, Gender, Complexity = environment();
    n_i=length(unique(SubjectID));
    n_t=length(unique(DecisionID));
    n_s=length(unique(SessionID));
    n_i_per_s=Int64(n_i*n_t/n_s) 
    X = [Constant Treatment SwitchPoint Gender Complexity];
    β = [1.0; 0.75; 0.01; 0.0; -0.1]; n_β = length(β);
    # Dummy variable generation
    D1 = ifelse.(SessionID .== 1,1,0);
    D2 = ifelse.(SessionID .== 2,1,0);
    D3 = ifelse.(SessionID .== 3,1,0);
    D4 = ifelse.(SessionID .== 4,1,0);
    D5 = ifelse.(SessionID .== 5,1,0);
    D6 = ifelse.(SessionID .== 6,1,0);
    D7 = ifelse.(SessionID .== 7,1,0);
    D8 = ifelse.(SessionID .== 8,1,0);
    D9 = ifelse.(SessionID .== 9,1,0);
    D10 = ifelse.(SessionID .== 10,1,0);
    D11 = ifelse.(SessionID .== 11,1,0);
    D12 = ifelse.(SessionID .== 12,1,0);
    D13 = ifelse.(SessionID .== 13,1,0);
    D14 = ifelse.(SessionID .== 14,1,0);
    D15 = ifelse.(SessionID .== 15,1,0);
    D16 = ifelse.(SessionID .== 16,1,0);
    D17 = ifelse.(SessionID .== 17,1,0);
    D18 = ifelse.(SessionID .== 18,1,0);
    D19 = ifelse.(SessionID .== 19,1,0);
    D20 = ifelse.(SessionID .== 20,1,0);
    β̂_Mat = zeros(n_β+18,rep_time);# We have to erase two dummies (not one). One from before D10, one after D11. I choose D1 as instruction and D11 for the latter part. This happens because of the multicolinearity occurs also with Treatment variable if we have all dummies from D11 to D20.
    p_val_Mat_Dummy = zeros(n_β+18,rep_time); 
    Session_Effect_Mat = zeros(n_s,rep_time);
        for s = 1:rep_time
            ϵ = rand(Normal(0,7.5),n_i*n_t);
            Session_Effect = rand(Normal(0,7.5),n_s);
            Session_Effect_Mat[:,s] = Session_Effect;
            u = repeat(Session_Effect, inner=n_i_per_s);
            Choice = X * β + ϵ + u;
            df=(;Choice,Treatment, SwitchPoint, Gender, Complexity,
            D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D20); # We have to erase two dummies (not one). One from before , because of the multicolinearity
            lm_builtin = reg(df, @formula(Choice~Treatment+SwitchPoint+Gender+Complexity+
            D2+D3+D4+D5+D6+D7+D8+D9+D10+D12+D13+D14+D15+D16+D17+D18+D19+D20))
            β̂_Mat[:,s] = coef(lm_builtin);
            p_val_Mat_Dummy[:,s] = coeftable(lm_builtin).cols[4];     
        end
    β̂_Mat = vcat(β̂_Mat[size(β̂_Mat,1),:]',β̂_Mat)
    β̂_Mat = β̂_Mat[1:size(β̂_Mat,1)-1,:]
    p_val_Mat_Dummy = vcat(p_val_Mat_Dummy[size(p_val_Mat_Dummy,1),:]',p_val_Mat_Dummy)
    p_val_Mat_Dummy = p_val_Mat_Dummy[1:size(p_val_Mat_Dummy,1)-1,:]
    bias = median(β̂_Mat[1:5,:], dims = 2) .- β[1:5];# bias for normal variables
    power_β̂_Dummy = sum(p_val_Mat_Dummy[1:5,:].<0.05, dims=2)./rep_time #p value for normal variables
    power_Session_Effect = sum(p_val_Mat_Dummy[6:size(β̂_Mat,1),:].<0.05, dims=2)./rep_time # p value for dummies
    return β̂_Mat, bias, power_β̂_Dummy
end

@time β̂_Mat3e_manual, bias3e_manual, power3e_manual =Third_e_manual(1000);

println(DataFrame(bias = vec(bias3e_manual), power = vec(power3e_manual)))

  1.488022 seconds (6.20 M allocations: 1.482 GiB, 10.49% gc time)
5×2 DataFrame
 Row │ bias         power   
     │ Float64      Float64 
─────┼──────────────────────
   1 │ -0.256319      0.811
   2 │  0.573432      0.835
   3 │  0.344832      0.047
   4 │  0.00944414    0.059
   5 │  0.107108      0.71

As expected the power for the test of $\beta_{0}$ and $\beta_{1}$ is quite high because of the high variances of $\hat{\gamma}_{0}$ and $\hat{\gamma}_{1}$. Bias is also not very small. But for $\beta_{4}$, which is the only one in our parameters that the null is true, we can see that it is quite valid.